
# Антагонистическиие игры
---



In [126]:
import pandas as pd
import numpy as np
import fractions

## Численный метод Брауна-Робинсон

In [127]:
class Game:
    def __init__(self, game_matrix: np.array, mixed_1: np.array, mixed_2: np.array, mixed_cost):
        self.game_matrix = game_matrix
        self.mixed_1 = mixed_1
        self.mixed_2 = mixed_2
        self.mixed_cost = mixed_cost

        self.pure_1 = 0
        self.pure_cost_1 = - np.inf
        n, m = game_matrix.shape
        for i in range(n):
            tmp = np.min(game_matrix[i, : ])
            if tmp > self.pure_cost_1:
                self.pure_1 = i
                self.pure_cost_1 = tmp
 
        self.pure_2 = 0
        self.pure_cost_2 = np.inf
        for i in range(n):
            tmp = np.max(game_matrix[ : ,i ])
            if tmp < self.pure_cost_2:
                self.pure_2 = i
                self.pure_cost_2 = tmp

    def __str__(self):
        return "".join([
        "mixed strat of А: {}\n".format(self.mixed_1),
        "mixed strat of Б: {}\n".format(self.mixed_2),
        "pure strat of А: {}\n".format(self.pure_1),
        "pure strat of Б: {}\n".format(self.pure_2),
        "mixed cost: {}\n".format(self.mixed_cost),
        "pure cost of А: {}\n".format(self.pure_cost_1),
        "pure cost of Б: {}\n".format(self.pure_cost_2)],
        )

class BrownRobinsonMethod:
    def __init__(self, eps = 10**-3):
        self.eps = eps

    def solve(self, game_matrix: np.array):
        m, n = game_matrix.shape

        # Вектора смешанных стратегий
        x = np.zeros(m)
        y = np.zeros(n)

        # Вектора выигрыша 1 игрока и проигрыша 2 игрока
        win_1 = np.zeros(m)
        loss_2 = np.zeros(n)

        # Текущие стратегии 1 и 2 игроков; в начале можно и случайный выбор
        cur_st_1 = 0 # np.random.randint(m)
        cur_st_2 = 0 # np.random.randint(n)

        lower_bounds = []
        upper_bounds = []


        step = 0
        cur_eps = np.inf
        while cur_eps > self.eps:
            # Каждый из игроков исходит от выбора другого игрока на шаге
            # Игрок 1 среди выбора стратегии игрока 2 пытается найти наиболее выигрышную стратегию
            # Игрок 2 среди выбора стратегии игрока 1 пытается найти наименее проигрышную стратегию
            step += 1

            win_1 = np.add(win_1, game_matrix[ : ,cur_st_2])
            loss_2 = np.add(loss_2, game_matrix[cur_st_1, : ])

            x[cur_st_1] += 1
            y[cur_st_2] += 1

            lower_bound = np.divide(np.min(loss_2), step)
            upper_bound = np.divide(np.max(win_1), np.double(step))
  
            lower_bounds.append(lower_bound)
            upper_bounds.append(upper_bound)

            cur_eps = np.min(upper_bounds) - np.max(lower_bounds)

            cur_st_1 = np.argmax(win_1)
            cur_st_2 = np.argmin(loss_2)


        cost = max(lower_bounds) + np.divide(cur_eps, 2)

        # Собственно, считаем, с какой вероятностью выбираем каждую стратегию
        x = np.divide(x, step)
        y = np.divide(y, step)

        # Чистая - наиболее вероятная в векторе смешанных
        return Game(game_matrix, x, y, cost)

In [128]:
# Ввод: для считывания поэлементно, можно сначала создать обычный 2-мерный массив, а затем его перегнать в np.array
# Для генерации хорошо бы использовать np.random: можно сразу сгенерировать массив нужных размеров
# Для считывания из csv есть pandas

#### Метод для ввода пользователем матрицы весовых коэффициентов и названия стратегий с клавиатуры

In [129]:
def enterCoef(n,m):
    game_matrix = [ [0]*m for i in range(n) ]
    names_of_strategies = [0] * m
    
    for i in range(m):
        name = input(f"Введите название  {i+1} стратегии: ")
        if (name.isdigit() == False) and (len(name) != 0):
            names_of_strategies[i] = name
        else:
            print("Плохой ввод названия стратегии")
            raise ValueError
            
    for i in range(n):
        for j in range(m):
            num = float(input(f"Введите элемент строки {i+1} столбца {j+1}: "))
            if num > 0:
                game_matrix[i][j] = num
            else:
                raise ValueError
            
    return (np.array(game_matrix), names_of_strategies)

#### Метод для ввода пользователем названия стратегий с клавиатуры и случайная генерация матрицы весовых коэффициентов

In [130]:
def randomCoef(n,m):
    names_of_strategies = [0] * m
    
    for i in range(m):
        name = input(f"Введите название  {i+1} стратегии: ")
        if (name.isdigit() == False) and (len(name) != 0):
            names_of_strategies[i] = name
        else:
            print("Плохой ввод названия стратегии")
            raise ValueError      
            
    game_matrix = np.random.randint(1, 500, size = (n,m))
    
    return (game_matrix,names_of_strategies)

#### Метод для считывания из csv

In [131]:
def readFromScv(path = "data/data.csv"):
    main_matrix = pd.read_csv(path)
    game_matrix = None
    names_of_strategies = list(main_matrix.columns)
    
    if sum(main_matrix.isnull().sum()):
        print("Содержатся пропуски!")
        raise ValueError
    else:
        if sum(main_matrix.dtypes == object) > 0:
            print("Содержится строка в файле!")
            raise ValueError
        else:
            game_matrix = main_matrix.to_numpy()
            
    return (game_matrix, names_of_strategies)

In [132]:
games = {1:"Ввод с клавиатуры", 2:"Случайная генерация", 3:"Чтение из csv"}

print(f"Выберите один из 3 вариантов получения данных \n Доступные варианты получения данных: {games}")
print()

game_matrix = None
game_names = None

try: 
    choice = int(input("Введите число для получения данных (от 1 до 3): "))
    
    if (choice > 3) or (choice <= 0):
        print("Введите число от 1 до 3!")
    
    else:
        print(f"Выбранный вариант получения данных: {games[choice]}")
        
        if choice == 1:
            try:
                n = int(input("Введите кол-во строк: "))
                m = int(input("Введите кол-во столбцов: "))
                
                if (n > 0) and (m > 0):
                    game_matrix, game_names  = enterCoef(n,m)
                else:
                    print("Неправильное кол-во строк/столбцов!")
            except ValueError:
                print("Неправильный ввод!")
            except KeyboardInterrupt:
                 print("Прервано пользователем")
        elif choice == 2:
            try:
                n = int(input("Введите кол-во строк: "))
                m = int(input("Введите кол-во столбцов: "))
                
                if (n > 0) and (m > 0):
                    game_matrix, game_names  = randomCoef(n,m)
                else:
                    print("Неправильное кол-во строк/столбцов!")
            except ValueError:
                print("Неправильный ввод!")
            except KeyboardInterrupt:
                print("Прервано пользователем")
        else:
            try:
                game_matrix, game_names = readFromScv()
            except FileNotFoundError:
                print("Файл не найден!")
            except ValueError:
                print("Измените значения ячеек в файле!")
            except KeyboardInterrupt:
                print("Прервано пользователем")
            
except ValueError:
    print("Неправильный ввод!")
except KeyboardInterrupt:
    print("Прервано пользователем")

Выберите один из 3 вариантов получения данных 
 Доступные варианты получения данных: {1: 'Ввод с клавиатуры', 2: 'Случайная генерация', 3: 'Чтение из csv'}

Введите число для получения данных (от 1 до 3): 3
Выбранный вариант получения данных: Чтение из csv


In [133]:
print(f"Названия стратегий: {game_names}\nМатрица весовых коэффициентов: \n{game_matrix}")

Названия стратегий: ['Telephones', 'Notebooks', 'Tablets']
Матрица весовых коэффициентов: 
[[300 280 230]
 [200 180 130]
 [240 380 400]]


In [134]:
example = np.array([
  [300, 280, 230],
  [200, 180, 130],
  [240, 380, 400]
])

brm = BrownRobinsonMethod(10**-1)
sol = brm.solve(game_matrix)

print(sol)

mixed strat of А: [0.69465649 0.         0.30534351]
mixed strat of Б: [0.80916031 0.         0.19083969]
pure strat of А: 2
pure strat of Б: 0
mixed cost: 281.7251113231552
pure cost of А: 240
pure cost of Б: 300



## Смешанные стратегии игроков

In [135]:
df = pd.DataFrame([sol.mixed_1, sol.mixed_2], ["Игрок А", "Игрок Б"], game_names)
df

,Telephones,Notebooks,Tablets
Игрок А,0.694656,0.0,0.305344
Игрок Б,0.809160,0.0,0.190840


## Чистые оптимальные стратегии игроков